# Dependencies

In [4]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


# Importing SQL data

In [2]:
s='''
        SELECT 
        StudyID
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,Study_IsCommercial
        ,Study_IsDraft

        FROM [CPMS_BI].[dbo].[ODP_Study]

    '''

In [3]:
study_df = pd.read_sql(s, connection)
#pre-processing
data = study_df.fillna("not available")

C:\Users\busaji\AppData\Local\Temp\ipykernel_8620\3566467870.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [6]:
#combine text data into one column
data['Text'] = data['StudyTitle'] + " " + data['StudyResearchSummary'] + " "+ data['StudyInclusionCriteria']

In [ ]:
#drop unrequired fields

data_cln = data.drop(['StudyTitle','StudyResearchSummary','StudyInclusionCriteria',
            'StudyTitle',
          'StudyDesignType','StudyPhases_Concatenated','StudySettings_Concatenated',
          'StudyShortName',
          'StudyManagingSpecialty','StudyManagingSpecialty_PrimarySubSpecialty'], axis=1)

# RAG program - TEST

In [7]:
# Embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
def generate_embeddings(df, text_column):
    return model.encode(df[text_column].tolist(), convert_to_tensor=True)

text_column = 'Text'
embeddings = generate_embeddings(data, text_column)

# Convert to numpy for FAISS
embedding_matrix = np.array([embedding.numpy() for embedding in embeddings])

# Create FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\busaji\Documents\Projects\RAG-search\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\busaji\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

       StudyID                                         StudyTitle  \
3380      6874               Active surveillance of vaccine study   
43940    60678  Understanding the associations between reactog...   
46645    63534  A toolkit to facilitate the introduction of RS...   
48246    65187               Understanding Flu Vaccination Advice   
29200    43771  Vaccine uptake and immunity screening assessme...   

                                          StudyShortName  \
3380                Active surveillance of vaccine study   
43940  CONNECT: understanding associations with react...   
46645                Developing an RSV toolkit for LMICs   
48246                                              GUIDE   
29200                                              VISTA   

                                    StudyResearchSummary  \
3380                                                       
43940                                      not available   
46645                                      n

In [ ]:
# Search function
def search(query, k=200):
    query_embedding = model.encode([query], convert_to_tensor=True)
    query_embedding = np.array(query_embedding.numpy()).astype(np.float32)
    distances, indices = index.search(query_embedding, k)
    return indices, distances

# Example usage
query = "Find all research around vaccines or MRnA"
indices, distances = search(query)

# Retrieve matching records
matching_records = data.iloc[indices[0]]
print(matching_records)



       StudyID                                         StudyTitle  \
3380      6874               Active surveillance of vaccine study   
41556    58109  A Phase 3, randomized, active-controlled, obse...   
39719    56121  An experimental medicine study of Crimean–Cong...   
40388    56845  An experimental medicine study of influenza an...   
43940    60678  Understanding the associations between reactog...   
...        ...                                                ...   
29896    44608  A Phase I Study to assess the safety and immun...   
19097    31585  A Phase I Dose-Escalation Study to Evaluate th...   
23881    37350  Evaluating the effect of immunisation with gro...   
9593     13763  Using P4 Peptide to Augment Ex Vivo Phagocyte ...   
3367      6861  An Opa protein vaccine against serogroup B men...   

                                          StudyShortName  \
3380                Active surveillance of vaccine study   
41556  COVID-19 mRNA vaccines in adults ≥18 years o

In [11]:
# Search function
def search(query, k=1000):
    query_embedding = model.encode([query], convert_to_tensor=True)
    query_embedding = np.array(query_embedding.numpy()).astype(np.float32)
    distances, indices = index.search(query_embedding, k)
    return indices, distances



# Mental health
query = "Find research around mental health"
indices, distances = search(query)

# Retrieve matching records
matching_records_ment_hlth = data.iloc[indices[0]]

In [ ]:
matching_records_ment_hlth